# Exportando um relatório em pdf da variações do preço de criptomoedas utilizando python

## Bibliotecas necessárias

Para fazer este projeto, foram utilizadas as seguintes bibliotecas do Python,

In [ ]:
# libs de análise e manipulação dos dados
import pandas as pd
import numpy as np